# ML Flow

Building Vector Assembler

In [0]:
from pyspark.ml.feature import VectorAssembler

# Prep the features (similar to before, but adding the new label)
feature_cols = ["total_views",  "v2p_rate",  "price_mean"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

ML Model (GBT)

In [0]:
import mlflow
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from mlflow.models.signature import infer_signature

# 1. Clean Split (Passing raw data)
X_train, X_test = df_final_prep.randomSplit([0.8, 0.2], seed=42)

# 2. Define the Candidates
classifiers = [
    LogisticRegression(labelCol="is_top_seller", featuresCol="features", maxIter=10),
    RandomForestClassifier(labelCol="is_top_seller", featuresCol="features", numTrees=20),
    GBTClassifier(labelCol="is_top_seller", featuresCol="features", maxIter=10)
]

# Use the volume path you defined earlier
dfs_tmp_path = "/Volumes/workspace/ecommerce/ecommerce_data"

# 3. The Tournament Loop
for classifier in classifiers:
    model_name = classifier.__class__.__name__
    
    with mlflow.start_run(run_name=model_name):
        # A. Create and Train Pipeline
        pipeline = Pipeline(stages=[assembler, classifier])
        model = pipeline.fit(X_train)
        
        # B. Make Predictions
        predictions = model.transform(X_test)
        
        # C. Generate Signature
        input_example = X_test.select(feature_cols).limit(5).toPandas()
        output_example = predictions.select("prediction").limit(5).toPandas()
        signature = infer_signature(input_example, output_example)
        
        # D. Calculate Metrics
        evaluator_auc = BinaryClassificationEvaluator(labelCol="is_top_seller", metricName="areaUnderROC")
        auc = evaluator_auc.evaluate(predictions)
        
        evaluator_f1 = MulticlassClassificationEvaluator(labelCol="is_top_seller", metricName="f1")
        f1 = evaluator_f1.evaluate(predictions)
        
        # E. Log everything to MLflow
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("f1_score", f1)
        
        # FIXED LINE: Added dfs_tmpdir to comply with Unity Catalog requirements
        mlflow.spark.log_model(
            model, 
            "model", 
            signature=signature, 
            dfs_tmpdir=dfs_tmp_path
        )
        
        print(f"🏆 {model_name} -> AUC: {auc:.4f}, F1: {f1:.4f}")

In [0]:
import mlflow

# 1. Identify the best run programmatically
best_run = mlflow.search_runs(
    order_by=["metrics.f1_score DESC"]
).iloc[0]

run_id = best_run.run_id
model_uri = f"runs:/{run_id}/model"
model_name = "workspace.ecommerce.elite_product_classifier"

print(f"🥇 Registering the {best_run['params.model_type']} (Run ID: {run_id})")

# 2. Register to Unity Catalog
model_version = mlflow.register_model(model_uri=model_uri, name=model_name)

# 3. (Optional) Set an alias so your 'Rising Stars' report always uses this version
from mlflow import MlflowClient
client = MlflowClient()
client.set_registered_model_alias(model_name, "champion", model_version.version)

print(f"✅ Success! Model is now live in Unity Catalog as '{model_name}' version {model_version.version}")